In [1]:
import pandas as pd
import numpy as np
import pickle 

In [2]:
data = pd.read_csv('t20_dataset.csv', index_col=0)

In [3]:
data

,batting_team,bowling_team,city,current_score,balls_left,wickets_left,crr,last_five,runs_x
31184,West Indies,Bangladesh,Mirpur,79,45,8,6.320000,25.0,132
28866,Pakistan,South Africa,Abu Dhabi,96,19,2,5.702970,13.0,119
30208,South Africa,New Zealand,Auckland,125,40,6,9.375000,59.0,165
8641,Pakistan,Australia,Dubai,72,62,10,7.448276,31.0,150
48889,Afghanistan,West Indies,Nagpur,53,54,6,4.818182,20.0,123
19013,Australia,England,Manchester,69,52,7,6.088235,22.0,145
44773,India,Australia,Adelaide,110,41,8,8.354430,38.0,188
19477,Australia,South Africa,Melbourne,119,47,8,9.780822,42.0,182
33151,West Indies,New Zealand,Pallekele,82,63,7,8.631579,40.0,139
1813,New Zealand,Australia,Sydney,61,44,5,4.815789,23.0,117


In [4]:
# spilting the data into training and testing 

X = data.drop(columns=['runs_x'])
y = data['runs_x']
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=1)

In [5]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor

In [6]:
trf = ColumnTransformer([
    ('trf',OneHotEncoder(sparse=False,drop='first'),['batting_team','bowling_team','city'])
]
,remainder='passthrough')

In [7]:
pipe = Pipeline(steps=[
    ('step1',trf),
    ('step2',StandardScaler()),
    ('step3',RandomForestRegressor(n_estimators = 700, random_state = 1))
    ])

In [8]:
pipe.fit(X_train, y_train)

Pipeline(steps=[('step1',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('trf',
                                                  OneHotEncoder(drop='first',
                                                                sparse=False),
                                                  ['batting_team',
                                                   'bowling_team', 'city'])])),
                ('step2', StandardScaler()),
                ('step3',
                 RandomForestRegressor(n_estimators=700, random_state=1))])

In [9]:
from sklearn.metrics import r2_score,mean_absolute_error
from sklearn.metrics import mean_squared_error 

y_pred = pipe.predict(X_test)
print(r2_score(y_test,y_pred))
print(mean_absolute_error(y_test,y_pred))
print(mean_squared_error(y_test, y_pred))

0.9803844881951274
2.0328873484944916
19.465188358311938


In [10]:
df = pd.DataFrame({'Y_TEST': y_test[:10], 'Y_PREDICT': y_pred[:10]})
df

,Y_TEST,Y_PREDICT
24568,147,146.987143
5691,139,139.180000
11871,157,156.752857
1430,115,115.000000
42056,231,231.012857
40686,119,121.544286
19392,156,154.815714
26754,126,126.624286
37670,152,156.602857
27336,129,129.165714


In [15]:
import plotly.express as px

fig = px.scatter(x=y_test, y=y_pred, color=y_test, title='Random Forest', width=700, height=500, )
fig.update_traces(marker_size=10)
fig.update_xaxes(title_text='REAL VALUE')
fig.update_yaxes(title_text='PREDICT VALUE')
fig.show()

In [17]:
pickle.dump(pipe,open('t20_randomForest_model.pkl','wb'))